In [22]:
import pandas as pd
import numpy as np


In [23]:
def beta(year: int, df_company: pd.DataFrame, df2: pd.DataFrame) -> float:
  dfc_year = df_company[df_company['Date'].dt.year == year].dropna()
  df2_year = df2[df2['Date'].dt.year == year].dropna()
  cor = pd.DataFrame(dfc_year['Change']).join(pd.DataFrame(df2_year['Change']), lsuffix='l', rsuffix='r', how='inner')
  corr = cor.corr().iloc[1, 0]
  std1 = cor['Changel'].std()
  std2 = cor['Changer'].std()
  return corr * (std1/std2)

def deviation(frame: pd.DataFrame, year: int) -> float:
  frame = frame[frame['Date'].dt.year == year]
  return frame['Change'].std()/100

def cost_of_debt(tax_rate: float, fininfo:pd.DataFrame, year: int) -> np.ndarray:
  fininfo = fininfo[fininfo.index == year]
  pretax_debt = np.abs(fininfo.iloc[:, 2]) / fininfo.iloc[:, 0] * 100
  return (pretax_debt * (1-tax_rate/100)).values[0]

def k(frame: pd.DataFrame, year: int) -> float:
  frame = frame[frame['Date'].dt.year == year]
  changes = frame['Change'].values/100 +1
  return (np.prod(changes) - 1)*100

def leverage(frame: pd.DataFrame, year: int) -> float:
  frame = frame[frame.index == year]
  return (frame.iloc[:, 0] / frame.iloc[:, 1]).values[0]

    
def leverage_industry(companies: list[pd.DataFrame], year: int) -> float:
  Ls= []
  for company in companies:
    L = leverage(company, year)
    Ls.append(L)
  return np.array(Ls).mean()


def k_CAPM(k_entity: float, risk_free: float, betta: float) -> float:
  return risk_free + betta*(k_entity-risk_free)

def Delta(leverage: float, COD: float, tax: float, risk_free: float) -> float:
  return leverage*(1-tax/100) * (risk_free - COD)

def k_CAPM2(k_entity: float, risk_free: float, betta: float, delta:float, lev: float, t: float|int) -> float:
  return risk_free + betta*(k_entity-risk_free) * (1+lev*(1-t/100)) + delta

In [24]:
Market = pd.read_excel("SPX.xlsx").dropna()
Industry = pd.read_excel("2021-2024_Chip_Industry_Returns.xlsx").dropna()
TSM_YIELD = pd.read_excel("2021-2024_TSM_Returns.xlsx").dropna()
INTEL = pd.read_excel("INTEL_DEBT-TO-EQUITY.xlsx", index_col=0).dropna()
BCOM = pd.read_excel("BROADCOM_DEBT-TO-EQUITY.xlsx", index_col=0).dropna()
NVD = pd.read_excel("NVIDIA_DEBT-TO-EQUITY.xlsx", index_col=0).dropna()
SAMS = pd.read_excel("SAMSUNG_DEBT-TO-EQUITY.xlsx", index_col=0).dropna()
QCOM = pd.read_excel("QUALCOMM_DEBT-TO-EQUITY.xlsx", index_col=0).dropna()
taxes = pd.read_csv("weighted_corporate_taxes.csv", index_col=0).dropna()
TSM = pd.read_excel('tsm_financial_information.xlsx', index_col=0).dropna()
tetr = pd.read_excel('effective_tax_rates_tsmc.xlsx', index_col=0).dropna()


In [25]:
kf_2021 = 1.52
kf_2022 = 3.88 
kf_2023 = 3.88 
kf_2024 = 4.58



In [26]:
k_tsm_2021 = k(TSM_YIELD, 2021)
k_tsm_2022 = k(TSM_YIELD, 2022)
k_tsm_2023 = k(TSM_YIELD, 2023)
k_tsm_2024 = k(TSM_YIELD, 2024)
k_us_2021 = k(Market, 2021)
k_us_2022 = k(Market, 2022)
k_us_2023 = k(Market, 2023)
k_us_2024 = k(Market, 2024)
k_ind_2021 = k(Industry, 2021)
k_ind_2022 = k(Industry, 2022)
k_ind_2023 = k(Industry, 2023)
k_ind_2024 = k(Industry, 2024)
k_us_2021, k_us_2022, k_us_2023, k_us_2024


(np.float64(28.303531979240248),
 np.float64(-19.437920612458072),
 np.float64(24.233434801403696),
 np.float64(23.29969165229384))

In [27]:
k_tsm_2021, k_tsm_2022, k_tsm_2023, k_tsm_2024

(np.float64(0.2542009814207935),
 np.float64(-37.02222965383163),
 np.float64(39.607349317852304),
 np.float64(89.91614458771046))

In [28]:
k_ind_2021, k_ind_2022, k_ind_2023, k_ind_2024

(np.float64(19.42411050522004),
 np.float64(-35.805143898538425),
 np.float64(65.02112657438623),
 np.float64(19.34867097913755))

In [29]:
b_market_tsm_2021 = beta(2021, TSM_YIELD, Market)
b_market_tsm_2022 = beta(2022, TSM_YIELD, Market)
b_market_tsm_2023 = beta(2023, TSM_YIELD, Market)
b_market_tsm_2024 = beta(2024, TSM_YIELD, Market)
b_ind_tsm_2021 = beta(2021, TSM_YIELD, Industry)
b_ind_tsm_2022 = beta(2022, TSM_YIELD, Industry)
b_ind_tsm_2023 = beta(2023, TSM_YIELD, Industry)
b_ind_tsm_2024 = beta(2024, TSM_YIELD, Industry)
b_market_tsm_2021, b_market_tsm_2022, b_market_tsm_2023, b_market_tsm_2024

(np.float64(0.3240734793326558),
 np.float64(1.2472246577199553),
 np.float64(1.5453811905848058),
 np.float64(0.29938433456211466))

In [30]:
b_ind_tsm_2021, b_ind_tsm_2022, b_ind_tsm_2023, b_ind_tsm_2024

(np.float64(0.593520890683248),
 np.float64(0.900943911800595),
 np.float64(1.0241834239090033),
 np.float64(0.8140325120277278))

In [31]:
std_tsm_2021 = deviation(TSM_YIELD, 2021)
std_tsm_2022 = deviation(TSM_YIELD, 2022)
std_tsm_2023 = deviation(TSM_YIELD, 2023)
std_tsm_2024 = deviation(TSM_YIELD, 2024)
std_us_2021 = deviation(Market, 2021)
std_us_2022 = deviation(Market, 2022)
std_us_2023 = deviation(Market, 2023)
std_us_2024 = deviation(Market, 2024)
std_ind_2021 = deviation(Industry, 2021)
std_ind_2022 = deviation(Industry, 2022)
std_ind_2023 = deviation(Industry, 2023)
std_ind_2024 = deviation(Industry, 2024)

In [32]:
std_tsm_2021, std_tsm_2022, std_tsm_2023, std_tsm_2024

(np.float64(0.035368756114555526),
 np.float64(0.14143543633586014),
 np.float64(0.10571415862283225),
 np.float64(0.072835237475976))

In [33]:
std_us_2021, std_us_2022, std_us_2023, std_us_2024

(np.float64(0.03178899466510669),
 np.float64(0.06632511293665927),
 np.float64(0.04245402772054696),
 np.float64(0.030577482307123066))

In [34]:
std_ind_2021, std_ind_2022, std_ind_2023, std_ind_2024

(np.float64(0.05312572509309089),
 np.float64(0.12092040206831779),
 np.float64(0.09210676734588556),
 np.float64(0.05287082815406781))

In [35]:
L_tsm_2021 = leverage(TSM, 2021)
L_tsm_2022 = leverage(TSM, 2022)
L_tsm_2023 = leverage(TSM, 2023)
L_tsm_2024 = leverage(TSM, 2024)
L_tsm_2021, L_tsm_2022, L_tsm_2023, L_tsm_2024

(np.float64(0.3512682862670756),
 np.float64(0.30528796723046425),
 np.float64(0.2855937996454409),
 np.float64(0.24539010373727121))

In [36]:
CD_tsm_2021 = cost_of_debt(20, TSM, 2021)
CD_tsm_2022 = cost_of_debt(20, TSM, 2022)
CD_tsm_2023 = cost_of_debt(20, TSM, 2023)
CD_tsm_2024 = cost_of_debt(20, TSM, 2024)
CD_tsm_2021, CD_tsm_2022, CD_tsm_2023, CD_tsm_2024

(np.float64(0.5730480089121984),
 np.float64(1.055256129747109),
 np.float64(0.973151947023681),
 np.float64(0.7995497533070943))

In [37]:
L_ind_2021 = leverage_industry([TSM, INTEL, BCOM, NVD, SAMS, QCOM], 2021)
L_ind_2022 = leverage_industry([TSM, INTEL, BCOM, NVD, SAMS, QCOM], 2022)
L_ind_2023 = leverage_industry([TSM, INTEL, BCOM, NVD, SAMS, QCOM], 2023)
L_ind_2024 = leverage_industry([TSM, INTEL, BCOM, NVD, SAMS, QCOM], 2024)
L_ind_2021, L_ind_2022, L_ind_2023, L_ind_2024


(np.float64(0.7518538852987499),
 np.float64(0.6406266139993696),
 np.float64(0.619016544548768),
 np.float64(0.4399021536878209))

In [38]:
D_2021 = Delta(L_tsm_2021, CD_tsm_2021, 20, 1.52)
D_2022 = Delta(L_tsm_2022, CD_tsm_2022, 20, 3.88)
D_2023 = Delta(L_tsm_2023, CD_tsm_2023, 20, 3.88)
D_2024 = Delta(L_tsm_2024, CD_tsm_2024, 20, 4.58)
D_2021, D_2022, D_2023, D_2024

(np.float64(0.26610736246928574),
 np.float64(0.6898882512769754),
 np.float64(0.6641422243531672),
 np.float64(0.7421480625676518))

In [39]:
ki_CAPM_market_2021 = k_CAPM(k_us_2021, 1.52, b_market_tsm_2021)
ki_CAPM_market_2022 = k_CAPM(k_us_2022, 3.88, b_market_tsm_2022)
ki_CAPM_market_2023 = k_CAPM(k_us_2023, 3.88, b_market_tsm_2023)
ki_CAPM_market_2024 = k_CAPM(k_us_2024, 4.58, b_market_tsm_2024)
ki_CAPM_ind_2021 = k_CAPM(k_ind_2021, 1.52, b_ind_tsm_2021)
ki_CAPM_ind_2022 = k_CAPM(k_ind_2022, 3.88, b_ind_tsm_2022)
ki_CAPM_ind_2023 = k_CAPM(k_ind_2023, 3.88, b_ind_tsm_2023)
ki_CAPM_ind_2024 = k_CAPM(k_ind_2024, 4.58, b_ind_tsm_2024)
ki_CAPM_market_2021, ki_CAPM_market_2022, ki_CAPM_market_2023, ki_CAPM_market_2024

(np.float64(10.199832397329839),
 np.float64(-25.20268555461411),
 np.float64(35.33381530588347),
 np.float64(10.184382428529965))

In [40]:
ki_CAPM_ind_2021, ki_CAPM_ind_2022, ki_CAPM_ind_2023, ki_CAPM_ind_2024

(np.float64(12.146463613949495),
 np.float64(-31.874088784318726),
 np.float64(66.49972835660863),
 np.float64(16.602178336458344))

In [41]:
ki_CAPM2_market_2021 = k_CAPM2(k_us_2021, 1.52, b_market_tsm_2021, D_2021, L_tsm_2021, 20)
ki_CAPM2_market_2022 = k_CAPM2(k_us_2022, 3.88, b_market_tsm_2022, D_2022, L_tsm_2022, 20)
ki_CAPM2_market_2023 = k_CAPM2(k_us_2023, 3.88, b_market_tsm_2023, D_2023, L_tsm_2023, 20)
ki_CAPM2_market_2024 = k_CAPM2(k_us_2024, 4.58, b_market_tsm_2024, D_2024, L_tsm_2024, 20)
ki_CAPM2_ind_2021 = k_CAPM2(k_ind_2021, 1.52, b_ind_tsm_2021, D_2021, L_tsm_2021, 20)
ki_CAPM2_ind_2022 = k_CAPM2(k_ind_2022, 3.88, b_ind_tsm_2022, D_2022, L_tsm_2022, 20)
ki_CAPM2_ind_2023 = k_CAPM2(k_ind_2023, 3.88, b_ind_tsm_2023, D_2023, L_tsm_2023, 20)
ki_CAPM2_ind_2024 = k_CAPM2(k_ind_2024, 4.58, b_ind_tsm_2024, D_2024, L_tsm_2024, 20)
ki_CAPM2_market_2021, ki_CAPM2_market_2022, ki_CAPM2_market_2023, ki_CAPM2_market_2024

(np.float64(12.905099640835523),
 np.float64(-31.61567246699387),
 np.float64(43.184369231479195),
 np.float64(12.026738479513863))

In [42]:
ki_CAPM2_ind_2021, ki_CAPM2_ind_2022, ki_CAPM2_ind_2023, ki_CAPM2_ind_2024

(np.float64(15.398762706619962),
 np.float64(-39.91643500115551),
 np.float64(81.47091550426518),
 np.float64(19.704425270331186))